## Preview Streaming data from MessageHub

In [ ]:
from pixiedust.display.app import *
from pixiedust.display.streaming.data import *
from pixiedust.display.streaming.bokeh import *
import requests

@PixieApp
class MessageHubStreamingApp():
    def setup(self):
        self.streamingDisplay = None
        self.streamingData = None
        self.contents = []
        self.schemaX = None      

    def newDisplayHandler(self, options, entity):
        if self.streamingDisplay is None:
            self.streamingDisplay = LineChartStreamingDisplay(options, entity)
        return self.streamingDisplay
    
    def getTopics(self):
        rest_endpoint = "https://kafka-rest-prod01.messagehub.services.us-south.bluemix.net:443"
        headers = {
            'X-Auth-Token': self.credentials["api_key"],
            'Content-Type': 'application/json'
        }        
        return requests.get('{}/topics'.format(rest_endpoint),headers = headers).json()
    
    @route()
    def mainScreen(self):
        return """
<div class="well" style="text-align:center">
    <div style="font-size:x-large">MessageHub Streaming Browser.</div>
    <div style="font-size:large">Click on a topic to start</div>
</div>

{%for topic in this.getTopics()%}
    {%if loop.first or ((loop.index % 4) == 1)%}
<div class="row">
    <div class="col-sm-2"/>
    {%endif%}
    <div pd_options="topic=$val(topic{{loop.index}}{{prefix}})" class="col-sm-2" style="border: 1px solid lightblue;margin: 10px;border-radius: 25px;cursor:pointer;
        min-height: 150px;background-color:#e6eeff;display: flex;align-items: center;justify-content:center">
        <span id="topic{{loop.index}}{{prefix}}">{{topic}}</span>
    </div>
    {%if loop.last or ((loop.index % 4) == 0)%}
    <div class="col-sm-2"/>
</div>
    {%endif%}
{%endfor%}
        """
    
    def displayNextTopics(self):
        payload = self.streamingData.getNextData()
        if payload is not None and len(payload)>0:
            self.contents = self.contents + payload
            self.contents = self.contents[-10:]
            print(self.contents)
            
    def newDisplayHandler(self, options, entity):
        if self.streamingDisplay is None:
            self.streamingDisplay = LineChartStreamingDisplay(options, entity)
        return self.streamingDisplay
            
    @route(topic="*",streampreview="*",schemaX="*")
    def showChart(self, schemaX):
        self.schemaX = schemaX
        self.streamingData.setSchemaX(self.schemaX)
        return """
        <h3>Real-time chart for {{this.schemaX}} (Average)</h3>
        <div pd_refresh_rate="1000" pd_entity="streamingData"></div>
        """
        
    
    @route(topic="*",streampreview="*")
    def createStreamWidget(self, streampreview):
        if streampreview=="realtimeChart":
            if self.schemaX is None:
                return """
  <h3>Display a real-time chart by selecting a column below</h3>
  <div>
    {%for key in this.streamingData.schema%}
    <div class="radio">
      <label>
          <input type="radio" pd_options="streampreview=""" + streampreview + """;schemaX=$val(schemaX{{loop.index}}{{prefix}})" 
              id="schemaX{{loop.index}}{{prefix}}" pd_target="targetstreaming{{prefix}}" 
              name="schemaX" value="{{key}}">{{key}}
      </label>
    </div>
    {%endfor%}
  </div>"""
            else:
                return """
                <h3>Real-time chart for {{this.schemaX}} (Average)</h3>
                <div pd_refresh_rate="1000" pd_entity="streamingData"></div>"""
        return """<div pd_refresh_rate="1000" pd_script="self.displayNextTopics()"></div>"""
        
    @route(topic="*")
    def previewTopic(self, topic):
        self.topic = topic
        if self.streamingData is not None:
            self.streamingData.close()
        self.streamingData = MessagehubStreamingAdapter( self.topic, self.credentials["username"], self.credentials["password"] )
        return """
<div class="row">
    <div class="col-sm-1">
        <input pd_options="streampreview=raw"
               pd_target="targetstreaming{{prefix}}" 
               type="button" value="Raw Data">
               
        <input pd_options="streampreview=realtimeChart"
               pd_target="targetstreaming{{prefix}}"
               pd_script="self.streamingDisplay=None"
               pd_refresh
               type="button" value="Real-time Chart">
    </div>
    <div class="col-sm-10" id="targetstreaming{{prefix}}" style="padding-left: 5px;margin-left: 25px;border-left: 1px gray solid;">
        <div pd_refresh_rate="1000">
            <pd_script>self.displayNextTopics()</pd_script>
        </div>
    </div>
</div>
        """
        
    
MessageHubStreamingApp().run(credentials={
    "username": "XXXX",
    "password": "XXXX",
    "api_key" : "XXXX"
})